### Chains

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### Simple Chain(기본 체인)

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser   # 출력 파서(문자열로 변환)

# PromptTemplate 생성
prompt = PromptTemplate(
    template="{country}의 수도는 어디인가요?",
    input_variables=['country']     # 사용자가 입력할 변수명 지정
)

#  LLM Model 생성
model = ChatOpenAI(
    model="gpt-5-nano",
    temperature=0
)

# OutputParser 생성
output_parser = StrOutputParser()   # 모델 출력 결과에서 텍스트 내용(content)만 뽑아주는 파서

# Chain 연결 ( 파이프(|) 연산자 사용 )
chain = prompt | model | output_parser

result = chain.invoke(input={"country" : '대한민국'})

print(result)

대한민국의 수도는 서울(서울특별시)입니다.


### Sequential Chain(순차 체인)

In [5]:
# 체인 1: 번역

from langchain_openai import ChatOpenAI
# 모델생성
llm = ChatOpenAI(model='gpt-5-nano', temperature=0)

input_text = """
The mood is right
The spirit's up
We're here tonight
And that's enough
Simply having a wonderful Christmastime
Simply having a wonderful Christmastime
The choir of children sing their song
They practiced all year long
Ding dong, ding dong
Ding dong, ding dong
Ding dong, ding dong
Dong dong, dong ding
The mood is right
The spirit's up
We're here tonight
Oh, and that's enough
We're simply having a wonderful Christmastime
We're simply having a wonderful Christmastime
Oh-oh-oh, Christmastime 
"""

# 번역 프롬프트 생성
trans_prompt = PromptTemplate(
    template="다음의 문장을 한글로 번역하세요 : {text}",
    input_variables=['text']
)

# 번역 체인 생성
output_parser = StrOutputParser()
translation_chain = trans_prompt | llm | output_parser

trans_output = translation_chain.invoke(input_text)
print(trans_output)

분위기가 좋다
기운이 들떠 있다
오늘 밤 우리는 여기 있다
그게 다야
그저 멋진 크리스마스 시간일 뿐이야
그저 멋진 크리스마스 시간일 뿐이야
아이들의 합창단이 노래를 부른다
그들은 일년 내내 연습해 왔다
딩동, 딩동
딩동, 딩동
딩동, 딩동
동동, 동딩
분위기가 좋다
기운이 들떠 있다
오늘 밤 우리는 여기 있다
오, 그게 다야
그저 멋진 크리스마스 시간일 뿐이야
그저 멋진 크리스마스 시간일 뿐이야
오오오, 크리스마스 시간


In [6]:
# 체인 2 : 요약 및 연결
from langchain_core.runnables import RunnableSequence   # 여러 Runnable(체인)을 순서대로 실행할 수 있음

# 요약 프롬프트 생성
sum_prompt = PromptTemplate(
    template="다음의 글을 짧게 요약하세요 : {text}",
    input_variables=['text']
)

# 요약 체인 생성
summary_chain = sum_prompt | llm

# 체인 연결
# 앞의 체인의 출력이 뒤의 체인 입력으로 자동 전달된다.
overall_chain = RunnableSequence(translation_chain, summary_chain)

final_output = overall_chain.invoke(input_text)
print(final_output)


content='크리스마스 분위기가 절정인 오늘 밤, 우리는 여기서 멋진 크리스마스 시간을 보내고 있다. 아이들 합창단이 오랜 연습 끝에 노래를 부르며 분위기를 더욱 밝힌다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1021, 'prompt_tokens': 198, 'total_tokens': 1219, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 960, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqrcgkyGD4JififenRaK2KLBwWMTU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b586e-d694-72e2-a6c0-af7f5622f1fe-0' usage_metadata={'input_tokens': 198, 'output_tokens': 1021, 'total_tokens': 1219, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 960}}


In [7]:
# 조건부 체인
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

llm = ChatOpenAI(model='gpt-5-nano', temperature=0)

# 평가 체인
grading_prompt = PromptTemplate(
    template="당신은 냉철한 평가자입니다. 아래 답변을 1~5점으로 평가해주세요:\n\n{text}",
    input_variables=['text']
)

grading_chain = grading_prompt | llm

# 기본 체인
default_prompt = PromptTemplate(
    template="당신은 사용자의 질문에 답하는 친절한 챗봇입니다.\n\n{text}",
    input_variables=['text']
)

default_chain = default_prompt | llm

def grading_routing_fn(input_dict) -> bool:
    text = input_dict.get('text', '')
    return text.strip().startswith('평가')

# RunnableBranch(
#     (조건함수, True일때 실행할 체인),
#     False일때 실행할 체인
# )
cond_chain = RunnableBranch(
    (grading_routing_fn, grading_chain),
    default_chain
)

# 일반 질문
print(cond_chain.invoke(input={'text':'인공지능이 무엇인가요?'}))

content='인공지능(AI)은 인간의 지능처럼 문제를 이해하고 판단해 실행하는 능력을 컴퓨터 시스템에 구현하는 기술과 학문 분야를 말합니다.\n\n주요 포인트\n- 목표: 언어 이해, 이미지 인식, 의사 결정, 학습 등 인간이 하는 일을 컴퓨터가 수행하도록 만드는 것.\n- 좁은 AI vs 일반 AI: 지금 대부분의 AI는 특정 작업에 특화된 “좁은 AI(Narrow AI)”이고, 인간처럼 모든 일을 수행하는 “일반 AI(AGI)”는 아직 없습니다.\n- 핵심 기술들:\n  - 머신러닝: 데이터로부터 패턴을 학습해 예측이나 분류를 수행\n  - 딥러닝: 많은 계층의 인공신경망을 사용해 복잡한 패턴을 더 잘 학습\n  - 강화학습: 환경과의 상호작용을 통해 최적의 행동을 학습\n  - 규칙 기반/전통적 AI: 사람이 만든 규칙에 따라 작동하는 방식도 있음\n- 작동 원리 간단히: 데이터로 모델을 만들고, 새로운 입력이 주어지면 그 모델이 가장 적합한 판단이나 예측을 내림.\n- 실생활 예시: 스마트폰 음성비서, 추천 시스템(영화/상품 추천), 스팸 필터, 자율주행 자동차, 얼굴 인식 등.\n\n주의점과 한계\n- 데이터 의존성: 데이터 품질과 편향에 따라 결과가 달라질 수 있음\n- 해석 가능성: 복잡한 모델은 왜 그렇게 판단하는지 설명하기 어려울 때가 있음\n- 안전·윤리: 프라이버시, 차별, 잘못된 사용 등의 문제 고려 필요\n\n필요하시면 더 자세한 예시나 원리(예: 머신러닝 vs 딥러닝 차이, how a chatbot works)도 설명해 드릴게요. 어떤 부분이 궁금한가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1203, 'prompt_tokens': 32, 'total_tokens': 1235, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_to

In [8]:
# 평가 질문
evaluation_input='평가: 인공지능(AI)은 인간의 지능처럼 문제를 이해하고 판단해 실행하는 능력을 컴퓨터 시스템에 구현하는 기술과 학문 분야를 말합니다.'
print(cond_chain.invoke(input={'text': evaluation_input}))

content='점수: 4점\n\n설명:\n- 장점: 인간의 지능이 수행하는 핵심 기능인 문제 이해, 판단/결정, 실행을 컴퓨터 시스템에 구현하는 기술‧학문이라는 고수준 정의가 잘 포착되어 있습니다.\n- 보완점: 지능의 범위를 다소 모호하게 제시하고 있어, 학습(데이터 기반 학습), 알고리즘(예: 머신러닝/추론), 인지적 기능의 구체성(감지, 추론, 의사결정, 자율성 등)이 빠져 있습니다. 더 구체적으로 확장하면 정의가 더욱 명확해질 것입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 730, 'prompt_tokens': 69, 'total_tokens': 799, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqtCmNRnl4y5c0D9nOK9xtw3UgoXa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b58cb-a7b9-7011-8e3a-4bef04d0c0c0-0' usage_metadata={'input_tokens': 69, 'output_tokens': 730, 'total_tokens': 799, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'o

In [9]:
# RunnablePassthrough 활용
# {'text': RunnablePassthrough()} : 들어온 문자열을 그대로 'text' 키의 값으로 할당
pass_chain = {'text': RunnablePassthrough()} | cond_chain

print(pass_chain.invoke('평가: 코끼리는 어류이다.'))

content='점수: 1점\n\n이유: "코끼리는 어류이다"는 과학적으로 잘못된 주장으로, 코끼리는 포유류이다. 따라서 평가 수준이 매우 낮다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 693, 'prompt_tokens': 40, 'total_tokens': 733, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CqtH9rjtfYVm30FRhIhTPjNuPNhEd', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b58cf-c715-7712-864d-419726365e90-0' usage_metadata={'input_tokens': 40, 'output_tokens': 693, 'total_tokens': 733, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 640}}
